In [25]:
line = "giajsglkadsjglkasdgjklsadgjlkdsgjklglka"
f = open('neural_net_score','a')
f.write("\n start:  %s | " %(str(datetime.datetime.now())))
f.write(line)
f.write(" | End %s" %(str(datetime.datetime.now())))
f.close()
